# Modelos de clasificación

En esta segunda parte del Test 1 de clasificación, nos centraremos en elegir los modelos que se adapten mejor al tipo de datos que hemos preparado en el primer script EDA, analizar sus resultados y justificar la elección en función de métricas clave de rendimiento.

Para obtener unos resultados óptimos en este problema de clasificación sin incurrir en tiempos de procesamiento desmedidos, probaremos a implementar los siguientes modelos:

· Regresión logística: Intentaremos obtener una referencia inicial de accuracy, precision, ROC-AUC y recall. Es un modelo rápido de entrenar y sencillo de interpretar, por lo que nos dará una idea inicial del comportamiento de los datos.

· Ramdom Forest: Entrenaremos un conjunto de árboles de decisión con muestras aleatorias del Dataset para intentar obtener una clasificación más profunda de los datos.

· K-Nearest Neighbors (KNN): Clasifica en base a los "vecinos" más cercanos. Es sencillo y fácil de interpretar, y para un dataset no demasiado extenso como es el caso, no es costoso computacionalmente.

· Support Vector Machines (SVM): Busca el hiperplano que mejor separa las clases en un espacio de características. Puede ser una de las mejores opciones para nuestro caso, ya que es muy efectivo en espacios de características con alta dimensionalidad y con clases bien definidas.

## Preparación de los datos

· One-hot encoding para las variables categóricas

· Separación en datos de entrenamiento y test

· Estandarización de las variables numéricas

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.svm import SVC
import joblib

In [2]:
# Cargamos el Dataset limpio
df = pd.read_csv('dataset_EDA.csv')

# Definimos características de la variable objetivo
X = df.drop(['symboling'], axis=1)
y = df['symboling']

# Definimos las columnas categóricas y numéricas para el preprocesamiento
categorical_columns = ['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 
                       'drive-wheels', 'engine-location', 'horsepower_binned', 'price_binned']

numeric_columns = ['normalized-losses', 'wheel-base', 'length', 'width', 'height', 
                   'curb-weight', 'engine-size', 'bore', 'stroke', 'compression-ratio', 
                   'horsepower', 'peak-rpm', 'city-mpg', 'highway-mpg', 'power_to_weight', 'make_price_ratio']

# Dividimos el conjunto de datos en entrenamiento (80%) y test (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creamos un transformador para aplicar One-hot encoding a las variables categóricas y estandarización a las numéricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', OneHotEncoder(), categorical_columns)
    ]
)

In [3]:
# Creamos el pipeline de preprocesamiento
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

## Entrenamiento y evaluación de los modelos

### Regresión logística

In [4]:
# Entrenamos el modelo de regresión logística
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train_preprocessed, y_train)

# Predicciones
y_pred_logreg = logreg.predict(X_test_preprocessed)

# Métricas de evaluación
print("Regresión Logística:")
print(classification_report(y_test, y_pred_logreg))
print("ROC-AUC:", roc_auc_score(y_test, logreg.predict_proba(X_test_preprocessed), multi_class='ovr'))

Regresión Logística:
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.83      1.00      0.91         5
           0       0.87      0.72      0.79        18
           1       0.56      0.62      0.59         8
           2       0.50      0.60      0.55         5
           3       0.75      0.75      0.75         4

    accuracy                           0.73        41
   macro avg       0.75      0.78      0.76        41
weighted avg       0.75      0.73      0.74        41

ROC-AUC: 0.9216106422628162


#### Conclusiones del modelo de regresión logística

La precisión y el recall promedio ponderados están entorno al 75%, lo que implica que el modelo tiene un buen desempeño general en la predicción de las clases correctas. Los valores -2, -1, 0 y 3 de nuestra variable objetivo 'symboling' son los que mejor se comportan frente al clasificador. Sin embargo, las otras dos clases tienen métricas relativamente bajas, lo que nos sugiere una dificultad del modelo para predecir correctamnete las características en esta clase.

El modelo tiene una exactitud de 73%, lo cual es razonable, pero sugiere que hay margen de mejora, especialmente en las clases menos pobladas.

El ROC-AUC es de 0.92, lo que implica un buen desempeño en la clasificación en términos generales.

La regresión logística muestra un buen rendimiento general, con una alta precisión y recall para algunas clases, pero es mejorable en la clasificación de otras.

### Random Forest

In [5]:
# Entrenamos el modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train_preprocessed, y_train)

# Predicciones
y_pred_rf = rf.predict(X_test_preprocessed)

# Métricas de evaluación
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))
print("ROC-AUC:", roc_auc_score(y_test, rf.predict_proba(X_test_preprocessed), multi_class='ovr'))

Random Forest:
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.83      1.00      0.91         5
           0       0.93      0.72      0.81        18
           1       0.54      0.88      0.67         8
           2       1.00      0.40      0.57         5
           3       0.80      1.00      0.89         4

    accuracy                           0.78        41
   macro avg       0.85      0.83      0.81        41
weighted avg       0.84      0.78      0.78        41

ROC-AUC: 0.9774502269067487


#### Conclusiones del modelo Random Forest

La precisión promedio ponderada es del 84%, lo que nos indica que el modelo tiene un buen comportamiento general en este caso. La métrica de recall también muestra valores muy altos para la mayoría de valores de la variable objetivo. Para ambas métricas, la clasificación alcanza el 100% de efectividad en la mayoría de las clases.

El F1-score ponderado es del 78%, lo que nos indica un equilibrio más que aceptable entre la precisión y el recall. Las clases -2, -1 y 3 nuevamente muestran un rendimiento elevado.

El modelo tiene una exactitud general del 78%, mejor que la ofrecida por el modelo anterior.

El ROC-AUC es del 0.9774, un resultado sobresaliente que nos indica que el modelo de Random Forest es muy eficaz en la clasificación general, con un rendimiento excelente a la hora de diferenciar entre las clases.

### K-Nearest Neighbors (KNN)

En primer lugar, creamos un preprocesador con el parámetro 'handle_unknown=ignore' para configurar el OneHotEncoder. Esto permite al codificador manejar categorías desconocidas durante la transformación sin generar errores. Además, incluimos un SimpleImputer para asegurar nuevamente que no haya valores nulos en las variables categóricas.

In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_columns),
        ('cat', Pipeline([
            # Incluimos SimpleImputer si es necesario
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_columns)
    ]
)

In [7]:
# Creamos el pipeline para KNN
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

# Definimos una cuadrícula de búsqueda para ajustar el número de vecinos
param_grid = {
    'classifier__n_neighbors': [3, 5, 7, 9, 11],  # Ajuste del número de vecinos
    'classifier__weights': ['uniform', 'distance']  # Probamos con diferentes pesos
}

# Usamos GridSearchCV para ajustar los hiperparámetros
grid_search_knn = GridSearchCV(knn_pipeline, param_grid, cv=5, scoring='accuracy')
grid_search_knn.fit(X_train, y_train)

# Mejor estimador encontrado
best_knn = grid_search_knn.best_estimator_

# Predicciones con el mejor KNN
y_pred_knn = best_knn.predict(X_test)

# Métricas de evaluación para KNN con el mejor modelo
print("K-Nearest Neighbors (KNN):")
print(classification_report(y_test, y_pred_knn, zero_division=0))

# ROC-AUC para KNN
knn_probabilities = best_knn.predict_proba(X_test)
print("ROC-AUC KNN:", roc_auc_score(y_test, knn_probabilities, multi_class='ovr'))

c:\Users\rtx11679\Documents\HybridIntelligence\MRL-TestDataScience-1\env1\Lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


K-Nearest Neighbors (KNN):
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.60      0.60      0.60         5
           0       0.83      0.56      0.67        18
           1       0.62      1.00      0.76         8
           2       0.29      0.40      0.33         5
           3       1.00      0.75      0.86         4

    accuracy                           0.66        41
   macro avg       0.72      0.72      0.70        41
weighted avg       0.72      0.66      0.66        41

ROC-AUC KNN: 0.8767596153465719


#### Conclusiones KNN

El modelo KNN tiene un rendimiento decente, pero inferior a los dos modelos anteriores. Aunque es capaz de predecir algunas clases con alta precisión, vemos dificultades para manejar otras, como los valores de symboling 0 y 2. Su exactitud general es del 66% y su ROC-AUC del 0.8767, valores aceptables pero que lo colocan en último lugar, por el momento, en terminos de rendimiento.

### Support Vector Machines (SVM)

In [8]:
# Creamos el pipeline para SVM
svm_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', SVC(probability=True, kernel='linear'))  # Usamos kernel 'linear' por simplicidad
])

# Entrenamos el modelo SVM
svm_pipeline.fit(X_train, y_train)

# Predicciones con SVM
y_pred_svm = svm_pipeline.predict(X_test)

# Métricas de evaluación para SVM
print("Support Vector Machine (SVM):")
print(classification_report(y_test, y_pred_svm))

# ROC-AUC para SVM
svm_probabilities = svm_pipeline.predict_proba(X_test)
print("ROC-AUC SVM:", roc_auc_score(y_test, svm_probabilities, multi_class='ovr'))

Support Vector Machine (SVM):
              precision    recall  f1-score   support

          -2       1.00      1.00      1.00         1
          -1       0.62      1.00      0.77         5
           0       0.87      0.72      0.79        18
           1       0.56      0.62      0.59         8
           2       0.50      0.40      0.44         5
           3       0.75      0.75      0.75         4

    accuracy                           0.71        41
   macro avg       0.72      0.75      0.72        41
weighted avg       0.72      0.71      0.71        41

ROC-AUC SVM: 0.9309365195234762


#### Conclusiones SVM

El modelo SVM ofrece un buen rendimiento, con una exactitud del 71% y un ROC-AUC de 0.9334. Aunque no supera los resultados del Random Forest, SVM se posiciona bien en comparación con la regresión logística y el KNN en términos de precisión, recall y ROC-AUC.

### Conclusiones finales

El modelo RANDOM FOREST es claramente el mejor en nuestro caso de estudio, con una exactitud del 78% y un ROC-AUC casi perfecto de 0.9774. Este modelo es capaz de manejar mejor las diferentes clases, ofreciendo un rendimiento superior en todas las métricas clave.

#### Serialización y guardado del modelo Random Forest

In [9]:
joblib.dump(rf, 'random_forest_model.pkl')

['random_forest_model.pkl']